### Lab: Connecting to Azure Cosmos DB Using Python SDK and Performing CRUD Operations

#### Objective:

Participants will learn how to connect to Azure Cosmos DB using the Python SDK and perform CRUD (Create, Read, Update, Delete) operations on a database, collections (containers), and documents.

### <span style="color: orange;">Install the required Azure Cosmos DB SDK</span>

In [ ]:
pip install azure-cosmos


### <span style="color: orange;">Setting Up CosmoDB Connection</span>

In [ ]:
from azure.cosmos import CosmosClient, exceptions

COSMOS_DB_URI = "https://localhost:8081"
COSMOS_DB_KEY = "C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw=="

# Initialize Cosmos Client
client = CosmosClient(COSMOS_DB_URI, COSMOS_DB_KEY)


### <span style="color: orange;">Create Database</span>

In [ ]:
def create_database(database_name):
    try:
        database = client.create_database_if_not_exists(id=database_name)
        print(f"Database '{database_name}' created or already exists.")
        return database
    except exceptions.CosmosHttpResponseError as e:
        print(f"Error creating database: {e}")


database = create_database("NewDB")

### <span style="color: orange;">Create Container</span>

In [28]:
def create_container(database, container_name, partition_key):
    try:
        container = database.create_container_if_not_exists(
            id=container_name,
            partition_key=partition_key,
            offer_throughput=400  # Default RU/s
        )
        print(f"Container '{container_name}' created or already exists.")
        return container
    except exceptions.CosmosHttpResponseError as e:
        print(f"Error creating container: {e}")

In [ ]:
# Create a container with a valid partition key
container_name = "SampleContainer"
container = database.create_container_if_not_exists(
    id=container_name,
    partition_key={"paths": ["/category"], "kind": "Hash"},  # Correct partition key
    offer_throughput=400  # Standard RU/s
)
print(f"Container '{container_name}' created or already exists.")


### <span style="color: orange;">Insert Document</span>

In [30]:
def insert_document(container, document):
    try:
        response = container.create_item(body=document)
        print(f"Document inserted: {response}")
    except exceptions.CosmosHttpResponseError as e:
        print(f"Error inserting document: {e}")

In [ ]:
sample_document = {
    "id": "1",  # Unique ID
    "name": "Azure Cosmos DB Lab",
    "category": "Tutorial",
    "description": "A lab to perform CRUD operations with Cosmos DB.",
}
insert_document(container, sample_document)


### <span style="color: orange;">Query Document</span>

In [32]:
def query_documents(container, query):
    try:
        items = list(container.query_items(
            query=query,
            enable_cross_partition_query=True
        ))
        print(f"Query result: {items}")
        return items
    except exceptions.CosmosHttpResponseError as e:
        print(f"Error querying documents: {e}")


In [ ]:
query = "SELECT * FROM c WHERE c.category = 'Tutorial'"
query_documents(container, query)


### <span style="color: orange;">Update  Document</span>

In [34]:
def update_document(container, document_id, updated_data):
    try:
        document = container.read_item(item=document_id, partition_key="Tutorial")
        for key, value in updated_data.items():
            document[key] = value
        response = container.upsert_item(body=document)
        print(f"Document updated: {response}")
    except exceptions.CosmosHttpResponseError as e:
        print(f"Error updating document: {e}")


In [ ]:
updated_data = {"description": "Updated description for Cosmos DB Lab."}
update_document(container, document_id="1", updated_data=updated_data)


### <span style="color: orange;">Delete Document</span>

In [36]:
def delete_document(container, document_id, partition_key):
    try:
        container.delete_item(item=document_id, partition_key=partition_key)
        print(f"Document with ID '{document_id}' deleted.")
    except exceptions.CosmosHttpResponseError as e:
        print(f"Error deleting document: {e}")


In [ ]:
delete_document(container, document_id="1", partition_key="Tutorial")


### <span style="color: orange;">Delete Container</span>

In [38]:
def delete_container(database, container_name):
    try:
        database.delete_container(container_name)
        print(f"Container '{container_name}' deleted.")
    except exceptions.CosmosHttpResponseError as e:
        print(f"Error deleting container: {e}")


In [ ]:
delete_container(database, "SampleContainer")


### <span style="color: orange;">Delete Database</span>

In [40]:
def delete_database(database_name):
    try:
        client.delete_database(database_name)
        print(f"Database '{database_name}' deleted.")
    except exceptions.CosmosHttpResponseError as e:
        print(f"Error deleting database: {e}")


In [ ]:
delete_database("SampleDB")